In [41]:
from scripts.stats import vlr_stats
import pandas as pd
import json
import duckdb

In [42]:
data = vlr_stats('na', '30')

In [43]:
# Extract player segments from the data
player_data = data['data']['segments']

# Normalize the JSON to a flat structure, using the 'agents' list as a string
df = pd.json_normalize(player_data)

# Convert the 'agents' list into a string for better readability
df['agents'] = df['agents'].apply(lambda x: ', '.join(x))

# Convert numeric columns from string to float (if necessary)
numeric_columns = ['rounds_played', 'rating', 'average_combat_score', 'kill_deaths',
                   'average_damage_per_round', 'kills_per_round', 'assists_per_round',
                   'first_kills_per_round', 'first_deaths_per_round']

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

In [45]:
df.head()

,player,org,agents,rounds_played,rating,average_combat_score,kill_deaths,kill_assists_survived_traded,average_damage_per_round,kills_per_round,assists_per_round,first_kills_per_round,first_deaths_per_round,headshot_percentage,clutch_success_percentage
0,Kaoticcc,SaD,"jett, raze, neon",203,1.23,297.1,1.34,71%,182.6,1.03,0.18,0.28,0.19,27%,20%
1,Spaz,TACO,"omen, viper, neon",309,1.23,218.8,1.20,79%,135.2,0.77,0.44,0.10,0.06,22%,29%
2,melya,Elat,"omen, sova, clove",308,1.17,227.8,1.14,74%,148.2,0.79,0.44,0.06,0.04,30%,9%
3,welyy,Blue,"sova, fade, skye",444,1.15,204.8,1.18,75%,136.8,0.69,0.37,0.09,0.05,29%,15%
4,Fair,SCC,"raze, killjoy, yoru",208,1.15,253.7,1.26,72%,158.0,0.90,0.12,0.25,0.17,28%,11%


In [46]:
df.to_csv('vlr_data', index=False)

In [47]:
conn = duckdb.connect(database=':memory:')

In [48]:
#Top Players by Rating
conn.execute('SELECT player, org, agents, rating from df order by rating desc limit 10').df()

,player,org,agents,rating
0,Kaoticcc,SaD,"jett, raze, neon",1.23
1,Spaz,TACO,"omen, viper, neon",1.23
2,melya,Elat,"omen, sova, clove",1.17
3,welyy,Blue,"sova, fade, skye",1.15
4,Fair,SCC,"raze, killjoy, yoru",1.15
5,Kyu,AMB,"viper, breach, sova",1.14
6,Pa1nt,Blue,"neon, jett, raze",1.14
7,Paincakes,FAIM,"sova, deadlock, skye",1.13
8,Shondex,Blue,"cypher, killjoy, viper",1.12
9,koalanoob,M80,"yoru, omen, jett",1.10


In [49]:
#Find first kill to first death ratio
conn.execute('SELECT player, org, agents, first_kills_per_round/first_deaths_per_round as fk_fd_ratio from df order by fk_fd_ratio desc limit 10').df()

,player,org,agents,fk_fd_ratio
0,welyy,Blue,"sova, fade, skye",1.800000
1,Spaz,TACO,"omen, viper, neon",1.666667
2,k0rupt,NA,"fade, sova, deadlock",1.571429
3,Nora,SOL,"viper, breach, harbor",1.571429
4,melya,Elat,"omen, sova, clove",1.500000
5,ExciteMVP,WVU,"sova, skye",1.500000
6,Kaoticcc,SaD,"jett, raze, neon",1.473684
7,Fair,SCC,"raze, killjoy, yoru",1.470588
8,Pa1nt,Blue,"neon, jett, raze",1.400000
9,Kyu,AMB,"viper, breach, sova",1.333333


In [50]:
#Find how headshot_percentage related to ACS
conn.execute('SELECT player, org, average_combat_score, headshot_percentage from df order by average_combat_score desc, headshot_percentage desc limit 10').df()

,player,org,average_combat_score,headshot_percentage
0,Kaoticcc,SaD,297.1,27%
1,Pa1nt,Blue,254.2,29%
2,Fair,SCC,253.7,28%
3,sym,TSM,240.3,30%
4,Okeanos,BLIN,234.1,29%
5,koalanoob,M80,233.9,23%
6,canezera,TOR,228.7,30%
7,melya,Elat,227.8,30%
8,Spaz,TACO,218.8,22%
9,jakee,YFP,217.2,33%


In [52]:
#Find the top 10 players who play Omen
conn.execute('''SELECT 
        player,
        org,
        agents,
        rating,
        average_combat_score,
        headshot_percentage,
    FROM df
    WHERE agents like '%omen%'
    order by rating desc, average_combat_score desc, headshot_percentage desc
    LIMIT 10''').df()

,player,org,agents,rating,average_combat_score,headshot_percentage
0,Spaz,TACO,"omen, viper, neon",1.23,218.8,22%
1,melya,Elat,"omen, sova, clove",1.17,227.8,30%
2,koalanoob,M80,"yoru, omen, jett",1.10,233.9,23%
3,jakee,YFP,"astra, omen, viper",1.09,217.2,33%
4,gMd,TSM,"brimstone, omen, astra",1.09,211.2,29%
5,zander,M80,"omen, viper, brimstone",1.07,209.6,28%
6,gucc107,WVU,"omen, brimstone",1.03,210.2,28%
7,NMSRLM,TV,"astra, omen, cypher",1.00,196.2,38%
8,mummAy,FAIM,"astra, brimstone, omen",0.99,163.7,31%
9,rise,FLOP,"omen, kayo, cypher",0.90,170.3,22%


Link to Interactive Dashboard below:

https://public.tableau.com/views/ValorantDashboard/Dashboard1?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link